In [15]:
import psycopg2

import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from datetime import datetime
from dateutil import parser

In [16]:
sgf_df_time_filtered = pd.read_csv('input/sgd_samples_.csv')

In [17]:
sgf_df_time_filtered

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,347632,347632,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,sulphur,18.80,ppm
1,347630,347630,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,nitrate_n,2.44,ppm
2,347623,347623,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,ph,7.64,NaN
3,347624,347624,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,electrical_conductivity,0.57,mS/cm
4,347625,347625,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,ammonium,2.44,ppm
...,...,...,...,...,...,...,...,...
435739,362040,362040,CK134SGF0272,2023-05-05 16:48:14.660,1:2 Soil Volume Extract,magnesium,11.70,ppm
435740,362039,362039,CK134SGF0272,2023-05-05 16:48:14.660,1:2 Soil Volume Extract,calcium,33.60,ppm
435741,362038,362038,CK134SGF0272,2023-05-05 16:48:14.660,1:2 Soil Volume Extract,ammonium,3.83,ppm
435742,363060,363060,CK134SGF0272,2023-05-05 16:48:14.660,1:2 Soil Volume Extract,silicon__,9.13,ppm


In [18]:
sgf_df_time_filtered.analysis_name.value_counts()

analysis_name
1:2 Soil Volume Extract                            350816
Drip Water Analysis                                 74760
Raw irrigation water analysis (for fertigation)      7265
Soil Suction Tube Solution Analysis                  1626
% Organic Matter,1:2 Soil Volume Extract              779
1:2 Soil Volume Extract,Flouride Analysis             280
1:2 Soil Volume Extract,Aluminium Analysis            140
EC [Salts] Analysis,pH Analysis                        22
% Organic Matter                                       22
Drip Water Analysis,Flouride Analysis                  20
Chloride Analysis,pH Analysis,Sulphate Analysis         9
Sulphate Analysis,pH Analysis,Chloride Analysis         3
Aluminium Analysis                                      2
Name: count, dtype: int64

In [19]:
df_ = sgf_df_time_filtered[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/sgf_unit_per_chemical.csv")

In [20]:
df_ = df_.reset_index()

In [21]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/sgf_unit_per_chemical_decision.csv")

1:2 Soil Volume Extract
Drip Water Analysis
Raw irrigation water analysis (for fertigation)
Soil Suction Tube Solution Analysis
% Organic Matter,1:2 Soil Volume Extract
% Organic Matter
1:2 Soil Volume Extract,Flouride Analysis
1:2 Soil Volume Extract,Aluminium Analysis
EC [Salts] Analysis,pH Analysis
Aluminium Analysis
Drip Water Analysis,Flouride Analysis


In [22]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = sgf_df_time_filtered.loc[(sgf_df_time_filtered['analysis_name'] == analysis) & (sgf_df_time_filtered['chemical_name'] == chemical) & (sgf_df_time_filtered['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [23]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
9,347627,347627,CE032SGF1242,2023-03-27 09:04:50.000,1:2 Soil Volume Extract,magnesium,5.79,ppm
280,348875,348875,CB110SGF0084,2023-03-28 10:09:22.667,1:2 Soil Volume Extract,magnesium,13.30,ppm
330,348692,348692,CB110SGF0088,2023-03-28 10:09:22.667,1:2 Soil Volume Extract,magnesium,4.81,ppm
343,348861,348861,CB110SGF0082,2023-03-28 10:09:22.667,1:2 Soil Volume Extract,magnesium,31.80,ppm
345,348655,348655,CB110SGF0087,2023-03-28 10:09:22.667,1:2 Soil Volume Extract,magnesium,0.51,ppm
...,...,...,...,...,...,...,...,...
435170,377800,377800,CM173SGF1947,2023-05-02 12:44:00.000,Raw irrigation water analysis (for fertigation),electrical_conductivity,0.39,mS cm -1
333316,89923,89923,CG019SGF0253,2020-07-14 15:59:00.000,Raw irrigation water analysis (for fertigation),ph,7.32,cfu/100 ml
333472,90356,90356,CG019SGF0254,2020-07-14 15:59:00.000,Raw irrigation water analysis (for fertigation),ph,7.83,cfu/100 ml
333738,233359,233359,CB110SGF0036,2020-07-15 12:58:00.000,Raw irrigation water analysis (for fertigation),ph,6.61,cfu/100 ml


In [24]:
df.to_csv("output/sgf_analysis_cleaned.csv",chunksize=1000)